## Code to combine and clean and filter folder of csv files

### Step 1: Make the *_combined.csv file

In [ ]:
import pandas as pd
import os

# Configuration
input_directory = 'rpg/csv_working/'
consolidated_csv_path = 'rpg/rpg_small.csv'

filtered_subreddits = [
    "AskGameMasters",
    "DMAcademy",
    "dndhorrorstories",
    "DungeonMasters",
    "lfg",
    "rpg",
    "rpghorrorstories"
]

dataframes = []

for filename in os.listdir(input_directory):
    if filename.endswith(".csv") and any(subreddit.lower() in filename.lower() for subreddit in filtered_subreddits):
        filepath = os.path.join(input_directory, filename)
        df = pd.read_csv(filepath)
        
        # Determine the source type and create 'text' column accordingly
        if '_comments' in filename:
            # For comments, use 'body' as the text
            df['text'] = df['body'].fillna('')
        else:
            # For submissions, concatenate 'title' and 'selftext'
            df['title'] = df['title'].fillna('')
            df['selftext'] = df['selftext'].fillna('')
            df['text'] = df['title'] + ' ' + df['selftext']
        
        dataframes.append(df)

# Concatenate all DataFrames
combined_df = pd.concat(dataframes, ignore_index=True)

# Save the combined DataFrame with the 'text' column to a new CSV file
combined_df.to_csv(consolidated_csv_path, index=False)

print(f"Combined DataFrame with 'text' column saved to {consolidated_csv_path}. Total rows: {len(combined_df)}")


### Step 2: Preprocess the *combined.csv

In [ ]:
import pandas as pd
import os
from html import unescape
import re

# Configuration
consolidated_csv_path = 'rpg/rpg_small.csv'
preprocessed_csv_path = 'rpg/rpg_small_preprocessed.csv'

def preprocess_text(text):
    """Enhanced text data preprocessing, returning an empty string for unsuitable content."""
    if pd.isna(text) or re.match(r'^\s*$', str(text)):
        return ''
    text = unescape(text)
    text = re.sub(r"http\S+|www\.\S+", "", text, flags=re.IGNORECASE)
    text = re.sub(r"â€™|‘|’", "'", text)
    text = re.sub(r'“|”', '"', text)
    text = re.sub(r"[^a-zA-Z0-9\s,.!?':;\"-]", "", text)
    text = re.sub(r'<[^>]*>', '', text)
    text = re.sub(r'&[a-zA-Z]+;', '', text)
    text = ' '.join(text.split())
    return text.strip()

# Load the combined DataFrame directly from the CSV
combined_df = pd.read_csv(consolidated_csv_path, dtype=str, low_memory=False)

# Remove rows with '[deleted]' or '[removed]' before any other preprocessing
patterns_to_remove = r'\[deleted\]|\[removed\]|\[deleted by user\]|\[removed by user\]'
for column in ['body', 'title', 'selftext']:
    if column in combined_df.columns:
        combined_df = combined_df[~combined_df[column].str.contains(patterns_to_remove, na=False, regex=True)]

# Apply preprocessing to text columns
for column in ['body', 'title', 'selftext']:
    if column in combined_df.columns:
        combined_df[column] = combined_df[column].fillna('').apply(preprocess_text)

# Create the 'text' column by combining 'title', 'selftext', and 'body'
combined_df['text'] = combined_df.apply(
    lambda row: ' '.join(row[col] for col in ['title', 'selftext', 'body'] if col in row and pd.notnull(row[col])).strip(),
    axis=1
)

# Drop rows with empty 'text' after preprocessing
preprocessed_df = combined_df[combined_df['text'] != '']

# Save the preprocessed DataFrame to a new CSV file
preprocessed_df.to_csv(preprocessed_csv_path, index=False)

print(f"Preprocessed DataFrame saved to {preprocessed_csv_path}. Total rows: {len(preprocessed_df)}")


### Filtering by: Score, Number of words, and Duplicate entries

In [4]:
import pandas as pd

# Define file paths
preprocessed_csv_path = 'rpg/rpg_small_preprocessed.csv'
processed_csv_path = 'rpg/rpg_small_processed.csv'
trimmed_save = 'rpg/rpg_small_trimmed.csv'
duplicate_save = 'rpg/rpg_small_duplicates.csv'

# Define score, length, and duplicate thresholds
score_threshold = 5
length_threshold = 10
duplicate_threshold = 2

# Load the preprocessed DataFrame
df = pd.read_csv(preprocessed_csv_path, low_memory=False)

# Ensure text column is a string to avoid any unexpected errors during processing
df['text'] = df['text'].astype(str)

# Save the original DataFrame before filtering
df_original = df.copy()

# Filter rows based on score and length thresholds
if score_threshold is not None:
    df = df[df['score'] >= score_threshold]
if length_threshold is not None:
    df = df[df['text'].str.len() >= length_threshold]

# Group by the 'text' column and filter groups with more than the specified number of identical texts
if duplicate_threshold is not None:
    duplicates = df.groupby('text').filter(lambda x: len(x) > duplicate_threshold)
else:
    duplicates = pd.DataFrame()

# Sort by 'text' to see the duplicates together
duplicates_sorted = duplicates.sort_values(by='text')

# Save the identified duplicates to a CSV for easier viewing
duplicates_sorted.to_csv(duplicate_save, index=False)

# Find the indices of rows to remove (duplicates)
indices_to_remove = duplicates_sorted.index

# Remove these rows from the DataFrame
df = df.drop(indices_to_remove)

# Save the trimmed rows to a separate file
if score_threshold is not None and length_threshold is not None:
    df_trimmed = df_original[(df_original['score'] < score_threshold) | (df_original['text'].str.len() < length_threshold)]
elif score_threshold is not None:
    df_trimmed = df_original[df_original['score'] < score_threshold]
elif length_threshold is not None:
    df_trimmed = df_original[df_original['text'].str.len() < length_threshold]
else:
    df_trimmed = pd.DataFrame()

# Remove the duplicates from the trimmed rows
if not df_trimmed.empty:
    indices_to_remove_trimmed = indices_to_remove[indices_to_remove.isin(df_trimmed.index)]
    df_trimmed = df_trimmed.drop(indices_to_remove_trimmed)

df_trimmed.to_csv(trimmed_save, index=False)

# Save the processed DataFrame
df.to_csv(processed_csv_path, index=False)

# Log information
print(f"Processed DataFrame saved to {processed_csv_path}.")
print(f"Total rows after filtering and removing duplicates: {len(df)}")
print(f"Trimmed rows saved to {trimmed_save}.")
print(f"Total trimmed rows: {len(df_trimmed)}")
print(f"Duplicate rows saved to {duplicate_save}.")
print(f"Total number of rows with at least {duplicate_threshold} duplicates: {len(duplicates_sorted)}")

Processed DataFrame saved to rpg/rpg_small_processed.csv.
Total rows after filtering and removing duplicates: 2129078
Trimmed rows saved to rpg/rpg_small_trimmed.csv.
Total trimmed rows: 7901230
Duplicate rows saved to rpg/rpg_small_duplicates.csv.
Total number of rows with at least 2 duplicates: 8255


### Step 3: Filtering by score

In [1]:
import pandas as pd

# Define file paths
preprocessed_csv_path = 'rpg/rpg_small_preprocessed.csv'
processed_csv_path = 'rpg/rpg_small_processed.csv'
trimmed_save = 'rpg/rpg_small_trimmed.csv'

# Define score and length thresholds
score_threshold = 5
length_threshold = 10

# Load the preprocessed DataFrame
df = pd.read_csv(preprocessed_csv_path, low_memory=False)

# Ensure text column is a string to avoid any unexpected errors during processing
df['text'] = df['text'].astype(str)

# Save the original DataFrame before filtering
df_original = df.copy()

# Filter rows based on score and length thresholds
if score_threshold is not None:
    df = df[df['score'] >= score_threshold]
if length_threshold is not None:
    df = df[df['text'].str.len() >= length_threshold]

# Save the processed DataFrame
df.to_csv(processed_csv_path, index=False)

# Save the trimmed rows to a separate file
if score_threshold is not None and length_threshold is not None:
    df_trimmed = df_original[(df_original['score'] < score_threshold) | (df_original['text'].str.len() < length_threshold)]
elif score_threshold is not None:
    df_trimmed = df_original[df_original['score'] < score_threshold]
elif length_threshold is not None:
    df_trimmed = df_original[df_original['text'].str.len() < length_threshold]
else:
    df_trimmed = pd.DataFrame()

df_trimmed.to_csv(trimmed_save, index=False)

# Log information
print(f"Processed DataFrame saved to {processed_csv_path}.")
print(f"Total rows after filtering: {len(df)}")
print(f"Trimmed rows saved to {trimmed_save}.")
print(f"Total trimmed rows: {len(df_trimmed)}")

Processed DataFrame saved to rpg/rpg_small_processed.csv.
Total rows after filtering: 2137333
Trimmed rows saved to rpg/rpg_small_trimmed.csv.
Total trimmed rows: 7901230


### Step 4: Compare the preprocessing steps against the 'combined.csv' and the preprocessing rules (e.g., did we drop the right amount?)

In [ ]:
import pandas as pd

# Configuration
consolidated_csv_path = 'G:/BERTopic/attachment/attach_combined.csv'
preprocessed_csv_path = 'G:/BERTopic/attachment/attach_preprocessed.csv'

# Load the original combined CSV
combined_df = pd.read_csv(consolidated_csv_path, low_memory=False)

# Load the preprocessed CSV
preprocessed_df = pd.read_csv(preprocessed_csv_path, low_memory=False)

# Patterns to search for in the text before any other preprocessing
patterns_to_remove = r'\[deleted\]|\[removed\]|\[deleted by user\]|\[removed by user\]'

# For each relevant column, check if it contains any of the keywords
# and create a mask for rows that should be removed based on the original columns
masks = []
for column in ['body', 'title', 'selftext']:
    if column in combined_df.columns:
        mask = combined_df[column].str.contains(patterns_to_remove, na=False, regex=True)
        masks.append(mask)

# Combine masks with logical OR to find any rows that match the criteria in any text column
combined_mask = pd.concat(masks, axis=1).any(axis=1)

# Log removed rows based on the combined mask before preprocessing
removed_rows_before = combined_df[combined_mask]

# Review the removed rows to ensure they match expectations before preprocessing
print("Sample of rows identified for removal based on original dataset:")
print(removed_rows_before.head())

# After preprocessing, check again for any missed rows with the keywords in the 'text' column
missed_rows_after = preprocessed_df['text'].str.contains(patterns_to_remove, na=False, regex=True)

# Review any rows that were missed during preprocessing
print("Sample of rows missed for removal in the preprocessed dataset:")
print(preprocessed_df[missed_rows_after].head())

# Count and compare
print(f"Original rows: {len(combined_df)}")
print(f"Rows after preprocessing: {len(preprocessed_df)}")
print(f"Identified for removal before preprocessing: {len(removed_rows_before)}")
print(f"Missed rows after preprocessing: {preprocessed_df[missed_rows_after].shape[0]}")
print(f"Expected rows after preprocessing (Original - Identified): {len(combined_df) - len(removed_rows_before)}")


### (Optional) Step 5: Identify and remove identical posts/comments - weekly discussion threads

In [2]:
import pandas as pd

# Define file paths
preprocessed_csv_path = 'rpg/rpg_small_processed.csv'
duplicate_save = 'rpg/rpg_small_duplicates.csv'
processed_csv_path = 'rpg/rpg_small_deduplicated.csv'

# Load the preprocessed DataFrame
df = pd.read_csv(preprocessed_csv_path, low_memory=False)

# Ensure text column is a string to avoid any unexpected errors during processing
df['text'] = df['text'].astype(str)

# Group by the 'text' column and filter groups with more than 2 identical texts
duplicates = df.groupby('text').filter(lambda x: len(x) > 2)

# Sort by 'text' to see the duplicates together
duplicates_sorted = duplicates.sort_values(by='text')

# Save the identified duplicates to a CSV for easier viewing
duplicates_sorted.to_csv(duplicate_save, index=False)

# Find the indices of rows to remove (duplicates)
indices_to_remove = duplicates_sorted.index

# Remove these rows from the original DataFrame
df_cleaned = df.drop(indices_to_remove)

# Save the cleaned DataFrame, now without the identified duplicates
df_cleaned.to_csv(processed_csv_path, index=False)

# Log information
print(f"Processed DataFrame saved to {processed_csv_path}.")
print(f"Total rows after removing duplicates: {len(df_cleaned)}")
print(f"Total number of rows with at least two duplicates: {len(duplicates_sorted)}")

Processed DataFrame saved to rpg/rpg_small_deduplicated.csv.
Total rows after removing duplicates: 2129078
Total number of rows with at least two duplicates: 8255


### Checking CSV Files

In [ ]:
import pandas as pd

def load_and_check_csv(csv_file_path):
    try:
        # Load the CSV file
        df = pd.read_csv(csv_file_path)
        
        # Print the number of rows in the CSV
        print(f"Total number of rows: {len(df)}")
        
        # Print the number of rows with 'deleted' or 'removed' in the 'text' column
        deleted_removed_count = df['text'].str.contains('deleted|removed', na=False).sum()
        print(f"Number of rows with 'deleted' or 'removed' in the 'text' column: {deleted_removed_count}")
        
        # Check for potential errors
        # Example check: Null values in the 'text' column
        null_values_count = df['text'].isnull().sum()
        print(f"Number of rows with null values in the 'text' column: {null_values_count}")
        
        # Example check: Unexpected data types (non-string) in the 'text' column
        non_string_count = df[df['text'].apply(lambda x: not isinstance(x, str))].shape[0]
        print(f"Number of rows with non-string values in the 'text' column: {non_string_count}")
        
        # Add more checks as needed based on the expected data format
        
    except Exception as e:
        print(f"An error occurred: {e}")

# Call the function with the path to your CSV file
load_and_check_csv("G:/BERTopic/attachment/attach_processed_length10.csv")


### Optional: Validating the dataset and checking datatype

In [ ]:
### ATTEMPT ONE - Does not dcomputer similarity.

import pandas as pd

# Replace 'your_csv_file.csv' with the actual file path
csv_file_path = 'G:/BERTopic/attachment/attach_processed_length10.csv'
df = pd.read_csv(csv_file_path, low_memory=False)

# Print each unique subreddit
for subreddit in df['subreddit'].unique():
    print(subreddit)

# Print the data type of each column formatted as a table
print("\nColumn Data Types:")
print(df.dtypes.to_frame('Data Type'))  # Convert dtypes Series to DataFrame for nicer display

# Display the first few rows of the 'text' column to get a sense of the content
print(df['text'].head())

# Drop rows containing 'deleted' or 'removed'
df = df[~df['text'].str.contains("deleted|removed", na=False)]

# After removal, print the number of rows left for confirmation
print(f"Remaining rows after removing 'deleted' or 'removed': {len(df)}")

# Check for rows that might be empty or contain only spaces
empty_or_spaces = df[df['text'].str.strip().eq("")]
print(f"Number of empty or only spaces rows: {len(empty_or_spaces)}")


## Creates a text and excel file that describes the dataset

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from openpyxl import Workbook
from openpyxl.drawing.image import Image
from openpyxl.utils.dataframe import dataframe_to_rows

# Configuration
csv_file_path = 'rpg/rpg_small_preprocessed.csv'
excel_file_path = 'rpg/analysis/rpg_small_descriptives.xlsx'
plot_save_path = 'rpg/analysis/plots/'

# Ensure the plot_save_path directory exists
if not os.path.exists(plot_save_path):
    os.makedirs(plot_save_path)

# Load dataset without specifying dtype
df = pd.read_csv(csv_file_path)

# Compute 'word_count'
df['word_count'] = df['text'].str.split().str.len()

# Define word count ranges
word_count_ranges = {
    'less_than_10': (df['word_count'] < 10),
    '10_to_49': ((df['word_count'] >= 10) & (df['word_count'] < 50)),
    '50_to_99': ((df['word_count'] >= 50) & (df['word_count'] < 100)),
    '100_to_199': ((df['word_count'] >= 100) & (df['word_count'] < 200)),
    '200_or_more': (df['word_count'] >= 200)
}

# Define score ranges
score_ranges = {
    'less_than_0': (df['score'] < 0),
    'equal_0': (df['score'] == 0),
    '1_plus': (df['score'] > 0),
    '5_plus': (df['score'] >= 5),
    '10_plus': (df['score'] >= 10),
    '20_plus': (df['score'] >= 20)
}

# Calculate the total number of rows for each word count range
word_count_totals = {key: value.sum() for key, value in word_count_ranges.items()}

# Convert the dictionary to a DataFrame
word_count_df = pd.DataFrame(list(word_count_totals.items()), columns=['Range', 'Count'])

# Function to generate plots
def plot_distribution(data, title, xlabel, filename):
    plt.figure(figsize=(8, 6))
    sns.histplot(data, bins=50, kde=True)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel('Frequency')
    plt.savefig(filename)
    plt.close()

# Create a new workbook
book = Workbook()

# Create a 'Combined' sheet
combined_sheet = book.create_sheet(title='Combined')

# Generate and save plots for 'score' and 'word_count' for the entire dataset
overall_score_filename = f"{plot_save_path}overall_score.png"
plot_distribution(df['score'], "Overall Score Distribution", 'Score', overall_score_filename)
overall_word_count_filename = f"{plot_save_path}overall_word_count.png"
plot_distribution(df['word_count'], "Overall Word Count Distribution", 'Word Count', overall_word_count_filename)

# Add images to the 'Combined' sheet at the desired location
score_img = Image(overall_score_filename)
word_count_img = Image(overall_word_count_filename)
combined_sheet.add_image(score_img, 'B2')
combined_sheet.add_image(word_count_img, 'B20')

# Create a sheet for bucketed distributions of word count
word_count_sheet = book.create_sheet(title='Word Count Distribution')
for row in dataframe_to_rows(word_count_df, index=False, header=True):
    word_count_sheet.append(row)
word_count_plot_filename = f"{plot_save_path}word_count_distribution.png"
plt.figure(figsize=(8, 6))
sns.barplot(x='Range', y='Count', data=word_count_df)
plt.title("Word Count Distribution")
plt.xlabel("Word Count Range")
plt.ylabel("Count")
plt.savefig(word_count_plot_filename)
plt.close()
word_count_plot_img = Image(word_count_plot_filename)
word_count_sheet.add_image(word_count_plot_img, 'B10')

# Create a sheet for bucketed distributions of score
score_sheet = book.create_sheet(title='Score Distribution')
score_totals = {key: value.sum() for key, value in score_ranges.items()}
score_df = pd.DataFrame(list(score_totals.items()), columns=['Range', 'Count'])
for row in dataframe_to_rows(score_df, index=False, header=True):
    score_sheet.append(row)
score_plot_filename = f"{plot_save_path}score_distribution.png"
plt.figure(figsize=(8, 6))
sns.barplot(x='Range', y='Count', data=score_df)
plt.title("Score Distribution")
plt.xlabel("Score Range")
plt.ylabel("Count")
plt.savefig(score_plot_filename)
plt.close()
score_plot_img = Image(score_plot_filename)
score_sheet.add_image(score_plot_img, 'B10')

# Group the data by 'subreddit' and calculate the mean score and word count
grouped_data = df.groupby('subreddit')[['score', 'word_count']].mean()

# Create sheets for each subreddit
for subreddit, data in grouped_data.iterrows():
    sheet = book.create_sheet(title=subreddit)
    subreddit_df = df[df['subreddit'] == subreddit]
    
    # Generate and save plots for 'score' and 'word_count' for the subreddit
    subreddit_score_filename = f"{plot_save_path}{subreddit}_score.png"
    plot_distribution(subreddit_df['score'], f"{subreddit} Score Distribution", 'Score', subreddit_score_filename)
    subreddit_word_count_filename = f"{plot_save_path}{subreddit}_word_count.png"
    plot_distribution(subreddit_df['word_count'], f"{subreddit} Word Count Distribution", 'Word Count', subreddit_word_count_filename)
    
    # Add images to the subreddit sheet at the desired location
    subreddit_score_img = Image(subreddit_score_filename)
    subreddit_word_count_img = Image(subreddit_word_count_filename)
    sheet.add_image(subreddit_score_img, 'B2')
    sheet.add_image(subreddit_word_count_img, 'B20')
    
    # Write descriptive statistics to the subreddit sheet
    sheet['A1'] = 'Score'
    sheet['A2'] = data['score']
    sheet['A3'] = 'Word Count'
    sheet['A4'] = data['word_count']

# Remove the default sheet
book.remove(book.active)

# Save the workbook
book.save(excel_file_path)

# Close the workbook
book.close()

C:\Users\snake\AppData\Local\Temp\ipykernel_2112\1805183992.py:19: DtypeWarning: Columns (2,4,5,8,9,10,11,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file_path)
